In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 


import parquet

We will use November data to analyse and walkthrough the data and try to see the relationships between data



In [55]:
nyc_df_raw = pd.read_parquet("../data/raw/")
nyc_df_raw.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-09-01 00:28:12,2022-09-01 00:36:22,1.0,2.10,1.0,N,100,239,1,9.0,3.0,0.5,1.8,0.00,0.3,14.60,2.5,0.0
1,1,2022-09-01 00:51:58,2022-09-01 01:14:43,1.0,8.70,1.0,N,161,243,2,27.5,3.0,0.5,0.0,0.00,0.3,31.30,2.5,0.0
2,1,2022-09-01 00:08:29,2022-09-01 00:26:29,1.0,8.30,1.0,N,138,233,1,25.0,3.0,0.5,2.0,6.55,0.3,37.35,2.5,0.0
3,2,2022-09-01 00:02:24,2022-09-01 00:09:39,1.0,1.32,1.0,N,238,166,2,7.5,0.5,0.5,0.0,0.00,0.3,8.80,0.0,0.0
4,2,2022-09-01 00:47:25,2022-09-01 00:56:09,1.0,2.94,1.0,N,161,226,2,10.5,0.5,0.5,0.0,0.00,0.3,14.30,2.5,0.0


Our target column would be price which we would predict based of the drop of location. But first we need to predicted estimated time and then feed that time as input to out model to predict estimated price.

### Lets first see the data

In [56]:
nyc_df_raw.columns.tolist()

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'airport_fee']

We will drop columns  `[ 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'airport_fee']` we wont be needing these to predict base fare amount as this are extra charges and would be calculated after the ride is done.

In [57]:
drop_columns = [ 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'airport_fee']

In [58]:
nyc_df = nyc_df_raw.drop(drop_columns, axis=1)
nyc_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount
0,1,2022-09-01 00:28:12,2022-09-01 00:36:22,1.0,2.10,1.0,N,100,239,1,9.0
1,1,2022-09-01 00:51:58,2022-09-01 01:14:43,1.0,8.70,1.0,N,161,243,2,27.5
2,1,2022-09-01 00:08:29,2022-09-01 00:26:29,1.0,8.30,1.0,N,138,233,1,25.0
3,2,2022-09-01 00:02:24,2022-09-01 00:09:39,1.0,1.32,1.0,N,238,166,2,7.5
4,2,2022-09-01 00:47:25,2022-09-01 00:56:09,1.0,2.94,1.0,N,161,226,2,10.5


In [12]:
nyc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16578210 entries, 0 to 16578209
Data columns (total 11 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
dtypes: datetime64[ns](2), float64(4), int64(4), object(1)
memory usage: 1.4+ GB


Datatype seems to be okay and we wont need to change the datatypes, as date time columns and numeric columns are in their respective datatypes

We will create a new column as the trip_time based on the tpep_pickup_datetime',
 'tpep_dropoff_datetime' column

In [59]:
nyc_df['trip_time'] = nyc_df['tpep_dropoff_datetime']-nyc_df['tpep_pickup_datetime']

In [60]:
nyc_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,trip_time
0,1,2022-09-01 00:28:12,2022-09-01 00:36:22,1.0,2.10,1.0,N,100,239,1,9.0,0 days 00:08:10
1,1,2022-09-01 00:51:58,2022-09-01 01:14:43,1.0,8.70,1.0,N,161,243,2,27.5,0 days 00:22:45
2,1,2022-09-01 00:08:29,2022-09-01 00:26:29,1.0,8.30,1.0,N,138,233,1,25.0,0 days 00:18:00
3,2,2022-09-01 00:02:24,2022-09-01 00:09:39,1.0,1.32,1.0,N,238,166,2,7.5,0 days 00:07:15
4,2,2022-09-01 00:47:25,2022-09-01 00:56:09,1.0,2.94,1.0,N,161,226,2,10.5,0 days 00:08:44


we need to convert it into minutes or seconds i think

In [61]:
nyc_df['trip_time'].head()

0   0 days 00:08:10
1   0 days 00:22:45
2   0 days 00:18:00
3   0 days 00:07:15
4   0 days 00:08:44
Name: trip_time, dtype: timedelta64[ns]

In [62]:
nyc_df['trip_time'] = nyc_df['trip_time'].dt.total_seconds() / 60

In [63]:
nyc_df['trip_time'].head()

0     8.166667
1    22.750000
2    18.000000
3     7.250000
4     8.733333
Name: trip_time, dtype: float64

we have converted the trip_time column to minutes, lets round off it to 2 decimals

In [64]:
nyc_df['trip_time']=nyc_df['trip_time'].round(2)
nyc_df['trip_time'].head()

0     8.17
1    22.75
2    18.00
3     7.25
4     8.73
Name: trip_time, dtype: float64

Lets extract day of the week and day of the month from the pickup date

In [65]:
nyc_df['day_week'] = nyc_df['tpep_pickup_datetime'].dt.day_name()
nyc_df['day_month'] = nyc_df['tpep_pickup_datetime'].dt.day

In [66]:
nyc_df[['day_month','day_week','tpep_pickup_datetime']].sample(5)

,day_month,day_week,tpep_pickup_datetime
11675643,14,Wednesday,2022-12-14 09:57:17
9181187,22,Tuesday,2022-11-22 17:48:07
10939666,7,Wednesday,2022-12-07 23:14:13
15982516,26,Thursday,2023-01-26 18:24:16
8783293,18,Friday,2022-11-18 13:39:44


In [43]:
nyc_df.columns.tolist()

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'trip_time',
 'day_week',
 'day_month']

In [67]:
nyc_df.isna().sum()

VendorID                      0
tpep_pickup_datetime          0
tpep_dropoff_datetime         0
passenger_count          569777
trip_distance                 0
RatecodeID               569777
store_and_fwd_flag       569777
PULocationID                  0
DOLocationID                  0
payment_type                  0
fare_amount                   0
trip_time                     0
day_week                      0
day_month                     0
dtype: int64

We can delete 'tpep_pickup_datetime','tpep_dropoff_datetime','passenger_count', 'RatecodeID','store_and_fwd_flag'

In [68]:
to_drop = [ 'tpep_pickup_datetime','tpep_dropoff_datetime','passenger_count', 'RatecodeID','store_and_fwd_flag','payment_type']
nyc_df = nyc_df.drop(to_drop,axis=1)

In [69]:
nyc_df.isnull().sum()

VendorID         0
trip_distance    0
PULocationID     0
DOLocationID     0
fare_amount      0
trip_time        0
day_week         0
day_month        0
dtype: int64

In [71]:
nyc_df.describe().astype(int)

,VendorID,trip_distance,PULocationID,DOLocationID,fare_amount,trip_time,day_month
count,16578210,16578210,16578210,16578210,16578210,16578210,16578210
mean,1,5,165,163,5,17,15
std,0,562,65,70,34533,46,8
min,1,0,1,1,-133391414,-1397,1
25%,1,1,132,113,7,7,8
50%,2,1,162,162,11,12,15
75%,2,3,234,234,18,20,23
max,6,389678,265,265,187444,10029,31


## Univariate Analysis

MemoryError: Unable to allocate 885. MiB for an array with shape (1, 116047470) and data type float64